# AI for Classifying Traffic Signals

In this notebook, I have created a CNN model to classify images of traffic signals. The signals can be categorized into the following types:
- Stop Signs
- Crosswalk Signs
- Traffic Lights
- Speed Limit Signs

The dataset used is published on [Kaggle](https://www.kaggle.com/datasets/andrewmvd/road-sign-detection).

**Model Architecture**:
- **Convolutional Layers**: 3 layers to extract features from images.
- **Fully Connected Layers**: 3 layers to make predictions.
- **Dropout Layers**: 2 layers added between the fully connected layers to mitigate overfitting.

**To Do**:
- Handle images of varying sizes; most images are larger than 256x256 pixels.
- Experiment with different loss functions and optimizers.
- Tune the model with different hyperparameters.

In [1]:
import os
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import cv2

In [2]:
def parseXML(xmlDir): 
    onlyfiles = [xmlDir + f for f in listdir(xmlDir) if isfile(join(xmlDir, f))]
    data = []
    for xmlfile in onlyfiles:
        tree = ET.parse(xmlfile)
        root = tree.getroot()
        obj = {
            'filename': root[1].text,
            'label': root[4][0].text
        }
        data.append(obj)
    return data

def encoder(value):
    if value == 'stop':
        return 0
    elif value == 'trafficlight':
        return 1
    elif value == 'speedlimit':
        return 2
    elif value == 'crosswalk':
        return 3

def decoder(value):
    if value == 0:
        return 'stop'
    elif value == 1:
        return 'trafficlight'
    elif value == 2:
        return 'speedlimit'
    elif value == 3:
        return 'crosswalk'

In [3]:
class MyDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.data_frame = dataframe
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):link
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0])
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.data_frame.iloc[idx, -1]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [4]:
def get_data():
    # Load your DataFrame
    data = pd.DataFrame(parseXML('/kaggle/input/road-sign-detection/annotations/'))
    
    data['label'] = data['label'].apply(encoder)

    train_df, val_df = train_test_split(data, test_size=0.3, random_state=2024, stratify=data['label'])
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
    root_dir = '/kaggle/input/road-sign-detection/images/'

    # Create training and validation datasets
    train_dataset = MyDataset(dataframe=train_df, root_dir=root_dir, transform=transform)
    val_dataset = MyDataset(dataframe=val_df, root_dir=root_dir, transform=transform)

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    return train_loader, val_loader


In [5]:
train_loader,val_loader = get_data()

In [6]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(64 * 32 * 32, 512)
        self.dropout1 = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(p=0.2)
        self.fc3 = nn.Linear(256, 4)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.dropout1(x) 
        x = self.fc2(x)
        x = self.dropout2(x) 
        x = self.fc3(x)
        return x

In [7]:
def train_model(model, train_loader,device,optimizer,criterion):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss
    
def evaluate_model(model,val_loader,device,optimizer,criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return (val_loss,correct/total)
        
def fit_model(model, train_loader, val_loader, num_epochs=15, learning_rate=0.0001, device='cuda'):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    print('Starting training. Loss function: CrossEntropyLoss. Optimizer: Adam.')

    for epoch in range(num_epochs):
        running_loss = train_model(model,train_loader,device,optimizer,criterion)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')
        val_loss, accuracy = evaluate_model(model,val_loader,device,optimizer,criterion)
        print(f'Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100 * accuracy}%')

In [8]:
model = MyModel()

In [9]:
fit_model(model, train_loader, val_loader)

Starting training. Loss function: CrossEntropyLoss. Optimizer: Adam.
Epoch [1/15], Loss: 2.788779240474105
Validation Loss: 1.099358233726687, Accuracy: 74.24242424242425%
Epoch [2/15], Loss: 0.9973144014016725
Validation Loss: 1.179124878305528, Accuracy: 74.62121212121212%
Epoch [3/15], Loss: 0.5481181785464286
Validation Loss: 0.8792727766041126, Accuracy: 78.78787878787878%
Epoch [4/15], Loss: 0.41270040087401866
Validation Loss: 0.9122170051042404, Accuracy: 79.16666666666666%
Epoch [5/15], Loss: 0.22360042929649354
Validation Loss: 0.7088042265839047, Accuracy: 77.27272727272727%
Epoch [6/15], Loss: 0.31818989468738434
Validation Loss: 0.9556999074088203, Accuracy: 68.56060606060606%
Epoch [7/15], Loss: 0.25712847446557136
Validation Loss: 1.1111644241544936, Accuracy: 60.22727272727273%
Epoch [8/15], Loss: 0.26873738477006553
Validation Loss: 1.2137085224191348, Accuracy: 78.78787878787878%
Epoch [9/15], Loss: 0.07038846132345497
Validation Loss: 0.8967465338193708, Accuracy: 78